In [1]:
import pandas as pd
import numpy as np
import pymysql as db

In [2]:
df = pd.read_csv(r"C:\Users\karth\Desktop\Data Science\DS Project\Police Post\traffic stops.csv")

C:\Users\karth\AppData\Local\Temp\ipykernel_24084\403397528.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\karth\Desktop\Data Science\DS Project\Police Post\traffic stops.csv")


In [3]:
df

,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,54,48,Black,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,DL56GW6568
65534,2020-02-15,12:14:00,Canada,F,18,35,Hispanic,Seatbelt,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,TN73EO7098
65535,2020-02-15,12:15:00,USA,M,27,41,Asian,Seatbelt,DUI,True,Frisk,Ticket,True,30+ Min,True,GJ33MX8328
65536,2020-02-15,12:16:00,Canada,F,49,63,Black,Seatbelt,Other,False,NaN,Warning,True,0-15 Min,True,KA24UZ8488


In [62]:
df.dropna(axis = 1, how = 'all', inplace = True)

df.fillna({
    'driver_gender': 'Unknown',
    'driver_age': df['driver_age'].mean(),
    'search_type' : 'None'
}, inplace = True)

In [59]:
connection = db.connect(
    host = "localhost",
    user = "Karthick",
    password = "262000",
)

cur = connection.cursor()
cur.execute("create database Securecheck")

1

In [60]:
cur.execute("Use Securecheck;")

0

In [64]:
cur.execute("""
create table police_post
(
stop_date date,
stop_time time,
country_name varchar(50),
driver_gender varchar(50),
driver_age_raw int,
driver_age int,
driver_race varchar(15),
violation_raw varchar(20),
violation varchar(20),
search_conducted varchar(15),
search_type varchar(20),
stop_outcome varchar(20),
is_arrested varchar(10),
stop_duration varchar(25),
drugs_related_stop varchar(10),
vehicle_number varchar(20)
)
"""
)

0

In [65]:
print(df.isnull().sum())

stop_date             0
stop_time             0
country_name          0
driver_gender         0
driver_age_raw        0
driver_age            0
driver_race           0
violation_raw         0
violation             0
search_conducted      0
search_type           0
stop_outcome          0
is_arrested           0
stop_duration         0
drugs_related_stop    0
vehicle_number        0
dtype: int64


In [66]:
df = df.where(pd.notnull(df), None)

In [67]:
for index, row in df.iterrows():
    query = """
    INSERT INTO police_post (
        stop_date, stop_time, country_name, driver_gender, driver_age_raw,
        driver_age, driver_race, violation_raw, violation,
        search_conducted, search_type, stop_outcome, is_arrested,
        stop_duration, drugs_related_stop, vehicle_number
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    values = (
        row['stop_date'], 
        row['stop_time'], 
        row['country_name'], 
        row['driver_gender'],
        row['driver_age_raw'], 
        row['driver_age'], 
        row['driver_race'], 
        row['violation_raw'],
        row['violation'], 
        row['search_conducted'], 
        row['search_type'], 
        row['stop_outcome'],
        row['is_arrested'], 
        row['stop_duration'], 
        row['drugs_related_stop'], 
        row['vehicle_number']
    )
    
    cur.execute(query, values)

In [68]:
connection.commit()
cur.close()
connection.close()